In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 8.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.5/314.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0


In [5]:
from openai import OpenAI

In [6]:
client = OpenAI(api_key=API_KEY)

In [7]:
from pathlib import Path

JSONPATH = Path("/Users/joregan/Playing/hsi/annotations/final_resolved")

In [26]:
SKIPPABLES = ["room"]

def process_item(item):
    utt_key = "utterance_type"
    if not utt_key in item["high_level"]:
        utt_key = "topic_name"
    if item["high_level"][utt_key] == "conversation_generic":
        return {}
    current = item["high_level"]["current_topic"]
    if current in SKIPPABLES:
        return {}
    snippet = item["snippet"]
    return {
        "topic:": current,
        "snippet": snippet
    }

In [93]:
import json

alldata = []
filepointer = []

for jsonfile in JSONPATH.glob("*.json"):
    stem = jsonfile.stem
    parts = stem.split("_")
    person = parts[1]
    room = parts[3]

    with open(jsonfile) as f:
        data = json.load(f)

    output = {
        "person": person,
        "room": room,
        "items": []
    }
    for item in data.keys():
        processed = process_item(data[item])
        if processed and processed != {}:
            output["items"].append(processed)
    alldata.append(output)
    filepointer.append(stem)

In [31]:
with open("/tmp/hsi.json", "w") as f:
    json.dump(alldata, f, indent=4)

In [32]:
test = alldata[0]

In [83]:
PROMPT = f"""
Below is some data from a conversation. The data is in JSON format.
The data contains a list of items, each with a topic and a snippet. The topic is the subject of the conversation, and the snippet is a short excerpt from the conversation.
Continue the conversation by writing a new snippet that is relevant to the topic. The new snippet should contain a reference to the topic and should be a natural continuation of the conversation.
The new snippet should be in the same format as the original snippets, and should be a short paragraph of text, of between 30 and 80 words.
There should only be one reference to the topic in the new snippet. It should be a natural part of the conversation, and should not be forced or awkward. It should be marked in boldface using markdown syntax. Only the determiner should be marked; boldface is marked by a single asterisk. The reference should be to the item itself, not to an abstract property related to the item: for example, in "the style of that painting", only "that painting" is a valid reference to the item: "the style" is not a valid reference to the item.
The reference to the topic should be suitable for instructing a person to point at it in a conversation. It can be a noun phrase, or a pronoun, but it should be clear and unambiguous. If the reference is a noun phrase, it should include a determiner, such as "this" or "that". Do not count articles ("the", "a", "an") as determiners, we are only interested in demonstrative determiners. If the reference is a pronoun, it should be clear what it refers to. Be sure to vary the references so there are results with each type of reference.
If another topic is mentioned in the new snippet and the spatial relationship is very clear, it is ok to reuse that spatial reference.
'If you want, I can stop by the kiosk and get *a newspaper* for tomorrow.' is a bad example for a snippet, because it does not contain to the item itself, but to another item of that category; also it includes an article "a" instead of a demonstrative determiner.
Create as many new snippets as there are items in the data.
```json
{test}
```
"""

In [35]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                { "type": "text", "text": PROMPT },
            ]
        }
    ]
)

In [36]:
response

ChatCompletion(id='chatcmpl-BRGKBf83FizBn2cuY40CfosbbuwGm', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\'topic:\': \'Book_28\', \'snippet\': "Yeah, I noticed **that novel** when I came in, and it seemed interesting. It must belong to someone who really enjoys detective stories, don\'t you think?"}\n```', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1745837307, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_90122d973c', usage=CompletionUsage(completion_tokens=52, prompt_tokens=1029, total_tokens=1081, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [43]:
response_text = response.choices[0].message.content

In [44]:
response_text

'```json\n{\'topic:\': \'Book_28\', \'snippet\': "Yeah, I noticed **that novel** when I came in, and it seemed interesting. It must belong to someone who really enjoys detective stories, don\'t you think?"}\n```'

In [53]:
def clean_response(response_text):
    response_text = response_text.replace("```json", "")
    response_text = response_text.replace("```", "")
    response_text = response_text.strip()
    # response_text = response_text.replace("'", '"')
    # try:
    #     return json.loads(response_text)
    # except json.JSONDecodeError:
    #     print(response_text)
    data = eval(response_text)
    return data

In [54]:
clean_response(response_text)

{'topic:': 'Book_28',
 'snippet': "Yeah, I noticed **that novel** when I came in, and it seemed interesting. It must belong to someone who really enjoys detective stories, don't you think?"}

In [55]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                { "type": "text", "text": PROMPT },
            ]
        }
    ]
)

In [56]:
response

ChatCompletion(id='chatcmpl-BRGSLwcAubUqXJVqMYgh5T77PwpTP', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n[\n    {"topic:": "furniture", "snippet": "I\'ll rearrange the chairs today, so **this setup** feels more welcoming."},\n    {"topic:": "furniture", "snippet": "Perhaps moving **that table** near the window would brighten up the room."},\n    {"topic:": "furniture", "snippet": "You know, **those stools** would be perfect at the kitchen counter."},\n    {"topic:": "furniture", "snippet": "Let\'s think about where **this bench** could go; maybe the hallway?"},\n    {"topic:": "Television_f9d000b1", "snippet": "Maybe we could move **the TV** to the opposite wall for a cozier setup."},\n    {"topic:": "Dog_Statue_1", "snippet": "I checked around, and **the statue** seems perfectly fine now."},\n    {"topic:": "Dog_Statue_1", "snippet": "I\'d love to know how **that piece** first caught your eye."},\n    {"topic:": "Dog_Sta

In [57]:
response_text = response.choices[0].message.content

In [58]:
clean_response(response_text)

[{'topic:': 'furniture',
  'snippet': "I'll rearrange the chairs today, so **this setup** feels more welcoming."},
 {'topic:': 'furniture',
  'snippet': 'Perhaps moving **that table** near the window would brighten up the room.'},
 {'topic:': 'furniture',
  'snippet': 'You know, **those stools** would be perfect at the kitchen counter.'},
 {'topic:': 'furniture',
  'snippet': "Let's think about where **this bench** could go; maybe the hallway?"},
 {'topic:': 'Television_f9d000b1',
  'snippet': 'Maybe we could move **the TV** to the opposite wall for a cozier setup.'},
 {'topic:': 'Dog_Statue_1',
  'snippet': 'I checked around, and **the statue** seems perfectly fine now.'},
 {'topic:': 'Dog_Statue_1',
  'snippet': "I'd love to know how **that piece** first caught your eye."},
 {'topic:': 'Dog_Statue_1',
  'snippet': 'And I was thinking, maybe **this statue** deserves a spot in the garden.'},
 {'topic:': 'Tennis_Racquet_4',
  'snippet': 'I think **this racquet** needs a new grip before 

In [60]:
esponse = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                { "type": "text", "text": PROMPT },
            ]
        }
    ]
)

In [65]:
esponse
esponse_text = esponse.choices[0].message.content

In [66]:
clean_response(esponse_text)

{'person': '4',
 'room': '222',
 'items': [{'topic': 'furniture',
   'snippet': 'We can try rearranging *the* furniture into a more open layout, which might make the room feel bigger.'},
  {'topic': 'furniture',
   'snippet': 'Maybe moving *that* couch closer to the window will make the room more inviting.'},
  {'topic': 'furniture',
   'snippet': "Let's keep *that* chair near the entrance as it seems to welcome guests quite nicely."},
  {'topic': 'furniture',
   'snippet': 'Sure, I can help you with rearranging *the* furniture now so everything is ready for when guests arrive.'},
  {'topic': 'Television_f9d000b1',
   'snippet': 'Well, *the* TV is mostly just background noise; I prefer engaging conversations with friends.'},
  {'topic': 'Dog_Statue_1',
   'snippet': 'Oh, *the* dog statue is just missing a small piece, but it still looks great.'},
  {'topic': 'Dog_Statue_1',
   'snippet': 'I think *that* dog statue can be repaired easily; we should look into it.'},
  {'topic': 'Dog_Stat

In [84]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                { "type": "text", "text": PROMPT },
            ]
        }
    ]
)

In [85]:
response


ChatCompletion(id='chatcmpl-BRGlWs58wDnCeyaEyxswix4IoJmmS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n[\n    {\n        "topic": "furniture",\n        "snippet": "Let\'s rearrange the layout again. I think moving **this table** closer to the window might brighten up the space."\n    },\n    {\n        "topic": "Television_f9d000b1",\n        "snippet": "Speaking of TV, can we at least turn **it** on for some background noise while we chat?"\n    },\n    {\n        "topic": "Dog_Statue_1",\n        "snippet": "I\'m curious to know how **this statue** ended up here in the first place. It\'s quite a conversation piece!"\n    },\n    {\n        "topic": "Tennis_Racquet_4",\n        "snippet": "Oh, and I think **that racket** might come in handy if anyone wants to play later."\n    },\n    {\n        "topic": "Laptop_4",\n        "snippet": "By the way, I was wondering if **this laptop** has enough storage for my project fi

In [86]:
response_text = response.choices[0].message.content

In [77]:
clean_response(response_text)

[{'topic:': 'furniture',
  'snippet': "Every time I walk in here, it's like a whole new space! Can you remind me why you wanted to move *this furniture* all the way to the other side?"},
 {'topic:': 'Television_f9d000b1',
  'snippet': 'I understand, but did you catch what happened with *that TV* last week? I heard there was some issue with the signal.'},
 {'topic:': 'Dog_Statue_1',
  'snippet': "I hope *that dog statue* isn't too fragile, especially considering how easily things get knocked over in here."},
 {'topic:': 'Tennis_Racquet_4',
  'snippet': "Maybe after we're done, we can go out and hit a few balls with *this racket*. It's been a while since I've played tennis."},
 {'topic:': 'Laptop_4',
  'snippet': "Even though this looks like my old one, I'm sure *that laptop* belongs to you. Maybe we can check the files to confirm?"},
 {'topic:': 'KeyChain_76c05fe6',
  'snippet': 'You must have a lot of responsibilities with all those keys. I wonder if *that keychain* has any special mea

In [ ]:
clean_response(response_text)

{'person': '4',
 'room': '222',
 'items': [{'topic:': 'furniture',
   'snippet': "The new setup actually makes *this room* feel more spacious, don't you think?"},
  {'topic:': 'furniture',
   'snippet': "I can't quite remember, but wasn't *that couch* part of a set? It looks distinct on its own now."},
  {'topic:': 'furniture',
   'snippet': "Could we move everything next weekend? I'd like to try a different layout for *the sofas* this time."},
  {'topic:': 'furniture',
   'snippet': "Let's make sure we don't forget the coasters when we adjust *those tables* around."},
  {'topic:': 'Television_f9d000b1',
   'snippet': "Maybe when we're done chatting, we can catch a new show on *that TV* together."},
  {'topic:': 'Dog_Statue_1',
   'snippet': 'Actually, *this dog statue* looks pretty charming. It adds character to the room.'},
  {'topic:': 'Dog_Statue_1',
   'snippet': "I heard you fixed it, so how's *the statue* holding up now?"},
  {'topic:': 'Dog_Statue_1',
   'snippet': '*This statu

In [87]:
clean_response(response_text)

[{'topic': 'furniture',
  'snippet': "Let's rearrange the layout again. I think moving **this table** closer to the window might brighten up the space."},
 {'topic': 'Television_f9d000b1',
  'snippet': 'Speaking of TV, can we at least turn **it** on for some background noise while we chat?'},
 {'topic': 'Dog_Statue_1',
  'snippet': "I'm curious to know how **this statue** ended up here in the first place. It's quite a conversation piece!"},
 {'topic': 'Tennis_Racquet_4',
  'snippet': 'Oh, and I think **that racket** might come in handy if anyone wants to play later.'},
 {'topic': 'Laptop_4',
  'snippet': 'By the way, I was wondering if **this laptop** has enough storage for my project files.'},
 {'topic': 'KeyChain_76c05fe6',
  'snippet': 'I think **this keychain** would be easier to spot if we hung it by the door.'},
 {'topic': 'Newspaper_990192bf',
  'snippet': 'Could you pass **that newspaper** over? I want to clip out the Sunday puzzle section.'},
 {'topic': 'Box_7d6b3e2c',
  'snip

In [88]:
PROMPT = f"""
You are given a JSON list of conversation items. Each item has a "topic" and a "snippet" (a short excerpt).

Your task is to write a **new snippet** for each topic:
- The new snippet must naturally continue the conversation and be **30–80 words** long.
- It must include exactly **one reference** to the **specific topic**, using a demonstrative determiner ("this", "that", "these", "those") or a clear pronoun.
- **Only** the determiner should be marked in bold using markdown (e.g., *this painting*).
- Do not refer to a general category (e.g., "a newspaper"); refer directly to the topic.
- References must sound natural, not forced. Vary between noun phrases and pronouns where possible.
- If another topic is clearly mentioned in relation to the main one, it’s acceptable to reuse spatial references.
- Create **one new snippet per item**.

Bad example:  
'If you want, I can stop by the kiosk and get *a newspaper* for tomorrow.' → Incorrect: refers to another item, uses an article instead of a demonstrative determiner.

---

```json
{test}
```
"""

In [89]:
esponse = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                { "type": "text", "text": PROMPT },
            ]
        }
    ]
)

In [90]:
esponse_text = esponse.choices[0].message.content

In [91]:
clean_response(esponse_text)

[{'topic:': 'furniture',
  'snippet': "Can you rearrange *these pieces* a bit more toward the window? It might make the room feel more open and enhance the overall look. Otherwise, it seems a bit crowded in here, and we can't fully appreciate the space."},
 {'topic:': 'furniture',
  'snippet': "I see what you mean. *This couch* would look much better by the window, creating a cozy nook. How about rearranging it next weekend together? It'll give us a fresh perspective."},
 {'topic:': 'furniture',
  'snippet': 'You know, by moving *those sofas* around, we might find the perfect setup for comfortable conversations. It’s worth experimenting until we get it just right.'},
 {'topic:': 'furniture',
  'snippet': "I'd appreciate your help to shift *them* around. With your eye for detail and my muscle, we can transform the whole setting into something more inviting."},
 {'topic:': 'Television_f9d000b1',
  'snippet': 'I totally agree. Sometimes just sitting and chatting is far more fulfilling tha

In [94]:
def get_prompt(item):
    test = json.dumps(item, indent=4)
    return f"""
You are given a JSON list of conversation items. Each item has a "topic" and a "snippet" (a short excerpt).

Your task is to write a **new snippet** for each topic:
- The new snippet must naturally continue the conversation and be **30–80 words** long.
- It must include exactly **one reference** to the **specific topic**, using a demonstrative determiner ("this", "that", "these", "those") or a clear pronoun.
- **Only** the determiner should be marked in bold using markdown (e.g., *this painting*).
- Do not refer to a general category (e.g., "a newspaper"); refer directly to the topic.
- References must sound natural, not forced. Vary between noun phrases and pronouns where possible.
- If another topic is clearly mentioned in relation to the main one, it’s acceptable to reuse spatial references.
- Create **one new snippet per item**.

Bad example:  
'If you want, I can stop by the kiosk and get *a newspaper* for tomorrow.' → Incorrect: refers to another item, uses an article instead of a demonstrative determiner.

---

```json
{test}
```
"""

In [95]:
all_results = {}

In [96]:
def process_item(itemid, max_retries=3):
    current = alldata[itemid]
    filename = filepointer[itemid]
    prompt = get_prompt(current)

    allreskey = f"{filename}__{itemid}"

    if not allreskey in all_results:
        all_results[allreskey] = {
            "person": current["person"],
            "room": current["room"],
            "filename": filename,
            "items": []
        }
    current_result = all_results[allreskey]

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            { "type": "text", "text": prompt },
                        ]
                    }
                ]
            )

            response_text = response.choices[0].message.content
            response_data = clean_response(response_text)
            if type(response_data) == dict:
                current_result["items"].append(response_data["items"])
            elif type(response_data) == list:
                current_result["items"].append(response_data)

        except Exception as e:
            print(f"⚠️ Error on attempt {attempt + 1}: {e}")
            if attempt == max_retries - 1:
                print("Max retries reached. Skipping this item.")
                return


In [ ]:
for itemid in range(len(alldata)):
    process_item(itemid)